In [1]:
from keras.datasets import mnist

# 载入MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [2]:
import tensorflow as tf

print("扩充前 x_train.shape:", x_train.shape)  # 返回 (60000, 28, 28)

# 图像维度扩充
x_train = tf.pad(x_train, [[0, 0], [2,2], [2,2]]) / 255

# 输出扩充后的图像尺寸
print("扩充后 x_train.shape:", x_train.shape)  # 返回 (60000, 32, 32)

扩充前 x_train.shape: (60000, 28, 28)
扩充后 x_train.shape: (60000, 32, 32)


In [3]:
# 扩充图像通道
x_train = tf.stack([x_train] * 3, axis=-1)

# 输出扩充后的图像维度
print("扩充后 x_train.shape:", x_train.shape)

扩充后 x_train.shape: (60000, 32, 32, 3)


In [4]:
# 将最后2000个图像作为验证集
x_val = x_train[-2000:]
y_val = y_train[-2000:]

# 使用除去验证集的部分作为训练集
x_train = x_train[:-2000]
y_train = y_train[:-2000]

In [5]:
  # 构造基础模型
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# 输出基础模型结构信息
base_model.summary()

58889256/58889256 [==============================] - 4s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                              

In [6]:
from tensorflow.keras import layers, Model, losses

# 将基础模型的输出展平
x = layers.Flatten()(base_model.output)

# 定义全连接层
x = layers.Dense(10, activation='softmax')(x)

# 构建完整的模型
head_model = Model(inputs=base_model.input, outputs=x)

# 配置训练参数
head_model.compile(optimizer='adam',
                   loss=losses.sparse_categorical_crossentropy,
                   metrics=['accuracy'])

In [7]:
# 训练模型
batch_size = 64
epochs = 10
history = head_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))

Epoch 1/10
907/907 [==============================] - 47s 36ms/step - loss: 0.4654 - accuracy: 0.8366 - val_loss: 0.1178 - val_accuracy: 0.9720
Epoch 2/10
907/907 [==============================] - 32s 36ms/step - loss: 0.0746 - accuracy: 0.9815 - val_loss: 0.0597 - val_accuracy: 0.9885
Epoch 3/10
907/907 [==============================] - 32s 35ms/step - loss: 0.0643 - accuracy: 0.9854 - val_loss: 0.0745 - val_accuracy: 0.9885
Epoch 4/10
907/907 [==============================] - 32s 35ms/step - loss: 0.0544 - accuracy: 0.9879 - val_loss: 0.0670 - val_accuracy: 0.9860
Epoch 5/10
907/907 [==============================] - 32s 35ms/step - loss: 0.0729 - accuracy: 0.9844 - val_loss: 0.0386 - val_accuracy: 0.9920
Epoch 6/10
907/907 [==============================] - 32s 35ms/step - loss: 0.0369 - accuracy: 0.9910 - val_loss: 0.0386 - val_accuracy: 0.9925
Epoch 7/10
907/907 [==============================] - 32s 35ms/step - loss: 0.0336 - accuracy: 0.9926 - val_loss: 0.0376 - val_accuracy:

In [8]:
# 选择后5000个测试样本
x_test_selected = x_test[5000:]
y_test_selected = y_test[-5000:]

# 对测试集进行图像维度扩充
x_test_selected = tf.pad(x_test_selected, [[0, 0], [2, 2], [2, 2]]) / 255

# 扩充图像通道
x_test_selected = tf.stack([x_test_selected] * 3, axis=-1)

# 对模型进行测试
test_loss, test_accuracy = head_model.evaluate(x_test_selected, y_test_selected, verbose=2)
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

# 打印测试结果
print("loss: {:.4f} - accuracy: {:.4f}".format(test_loss, test_accuracy, val_loss, val_accuracy))

print(val_loss)
print(val_accuracy)

157/157 - 2s - loss: 0.0234 - accuracy: 0.9954 - 2s/epoch - 12ms/step
loss: 0.0234 - accuracy: 0.9954
[0.11779681593179703, 0.059714045375585556, 0.07447613775730133, 0.06698527187108994, 0.03861960768699646, 0.038609012961387634, 0.03756086528301239, 0.059565551578998566, 0.09565198421478271, 0.03294599428772926]
[0.972000002861023, 0.9884999990463257, 0.9884999990463257, 0.9860000014305115, 0.9919999837875366, 0.9925000071525574, 0.9944999814033508, 0.9929999709129333, 0.9815000295639038, 0.9929999709129333]
